# Predict survival on the Titanic
In this Lab, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy

### Dataset
The dataset contains 891 observations of 12 variables:
* **PassengerId**: Unique ID for each passenger
* **Survived**: Survival (0 = No; 1 = Yes)
* **Pclass**: Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* **Name**: Name
* **Sex**: Sex
* **Age**: Age
* **Sibsp**: Number of Siblings/Spouses Aboard
* **Parch**: Number of Parents/Children Aboard
* **Ticket**: Ticket Number
* **Fare**: Passenger Fare
* **Cabin**: Cabin
* **Embarked** Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

In [ ]:
%pip install jupyterlab

: 

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

: 

In [ ]:
# imports
import warnings
warnings.filterwarnings('ignore')
# your code here

In [ ]:
titanic = # your code here
titanic.head()

In [ ]:
# print some info about the dataframe
# your code here

Looks like there are some Nan values, let's see how many for each column

In [ ]:
titanic.isnull().sum()

**Cabin** contains a lot of Nan values, we'll drop this column <br>
We'll replace the Nan values in **Age** with the age's median, and the ones in **Embarked** with **'S'**, which is the most frequent one in this column

In [ ]:
# your code here to drop Cabin
# check the fillna documentation: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html
titanic["Age"]= titanic["Age"].fillna(# your code here)
titanic["Embarked"].fillna(# your code here)
titanic.isnull().sum()

## Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
print ('survival rate =', titanic.Survived.mean())
(titanic.groupby('Survived').size()/titanic.shape[0]).plot(kind="bar",title="Label's Balance")


In [ ]:
# make a function to plot survival against passenger attribute
def survival_rate(column,t):
    df=pd.DataFrame()
    df['total']=titanic.groupby(column).size()
    df['survived'] = titanic.groupby(column).sum()['Survived']
    df['percentage'] = round(df['survived']/df['total']*100,2)
    print(df)

    df['survived'].plot(kind=t)
    df['total'].plot(kind=t,alpha=0.5,title="Survivors per "+str(column))
    plt.show()

In [ ]:
# Draw survival per Sex
survival_rate("Sex","barh")

In [ ]:
# Draw survival per Class
survival_rate("Pclass","barh")

In [ ]:
# Graph survived per port of embarkation
survival_rate("Embarked","bar")

In [ ]:
# Draw survived per Number of Parents/Children Aboard (Parch)
# your code here

In [ ]:
# Draw survived per Number of Siblings/Spouses Aboard (SibSp)
# your code here

## Model training

Some of the columns don't have predictive power, so let's specify which ones are included for prediction

In [ ]:
predictors = ["Pclass", "Sex", "Age", 'SibSp' ,'Parch', "Fare", "Embarked"]  

We need now to convert text columns in **predictors** to numerical ones

In [ ]:
for col in predictors: # Loop through all columns in predictors
    if titanic[col].dtype == 'object':  # check if column's type is object (text)
        titanic[col] = pd.Categorical(titanic[col]).codes  # convert text to numerical

titanic.head()

In [ ]:
# Split the data into a training set and a testing set. Set: test_size=0.3, random_state=1
# your code here

print ("train shape", X_train.shape, y_train.shape)
print ("test shape", X_test.shape, y_test.shape)

In [ ]:
# import LogisticRegression from: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# your code here
clf = LogisticRegression(random_state=1)
# your code here
train_score = # your code here
test_score = # your code here
print ('train accuracy =', train_score)
print ('test accuracy =', test_score)

Let's print the model's parameters

In [ ]:
coeff = pd.DataFrame()
coeff['Feature'] = X_train.columns
coeff['Coefficient Estimate'] = pd.Series(clf.coef_[0])
coeff.loc[len(coeff)]=['Intercept',clf.intercept_[0]]
print (coeff)

We now need to predict class labels for the test set. We will also generate the class probabilities

In [ ]:
# predict class labels for the test set
y_pred = # your code here
print (y_pred)

In [ ]:
# generate class probabilities : http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
y_probs = # your code here
print (y_probs)

As you can see, the classifier outputs two probabilities for each row. It's predicting a 1 (Survived) any time the probability in the second column is greater than 0.5. Let's visualize it all together.

In [ ]:
pred = pd.DataFrame({        
        "Survived_original": y_test,
        "Survived_predicted": y_pred,
        "Survived_proba": np.transpose(y_probs)[1]      
        })
pred["Comparison"]= pred.Survived_original ==pred.Survived_predicted
pred.head()

## Confusion matrix

In [ ]:
from sklearn import metrics
print (metrics.confusion_matrix(y_test, y_pred))
print (metrics.classification_report(y_test, y_pred))

As you can see, we can have the classification report for each class

## K-Fold Cross Validation

In [ ]:
# import cross_validation from: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
# your code here
clf = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(clf, titanic[predictors], titanic["Survived"], scoring='accuracy', cv=5)
## see model 
print(scores)
# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

When you are improving a model, you want to make sur that you are really doing it and not just being lucky. This is why it's good to work with cross validation instead of one train/test split.